# <font color="red">CSE499A - Assignment 01</font>

## Joy Kumar Ghosh - 2211424642
Section-17

### Install & Import Library

In [ ]:
%pip install qiskit
%pip install pylatexenc
%pip install qiskit-aer
%pip install qiskit-ibm-runtime

In [1]:
from qiskit import *
from qiskit import QuantumCircuit
from qiskit.visualization import plot_state_qsphere, plot_histogram, plot_bloch_multivector, array_to_latex
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit.quantum_info import Operator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from math import sqrt
import random
from collections import Counter
import time

In [2]:
QiskitRuntimeService.save_account(channel="ibm_quantum",
                                  token="11318f27c420947d9fcc114bdf0dc0103873ca807db5e77f68463decf6fef341ea99779afdbf5c13cc9d87c68cffb3c8011cb4a1a1165f10d62f87eaee3cee4f",
                                  overwrite=True,
                                  set_as_default=True)

## Question - 01
Build a 1-qubit circuit and initialize its state in superposition without using any gate. Qubits must be in different relative phases.\
Implement proper state vector visualization technique.\
Explain your understanding in detail.

### Construct the Circuit

In [ ]:
circuit = QuantumCircuit(1)

# probablity of a superposition state
probabilities = (1/sqrt(2), 1/sqrt(2))

# initializing circuit with probablity amplitudes
circuit.initialize(probabilities, 0)

circuit.draw('mpl')

### Run on Simulator for geting the state vector and qsphere

In [ ]:
simulator = AerSimulator()
circuit.save_statevector()
job = simulator.run(circuit)
result = job.result()

### Visualize Satte Vector

In [ ]:
final_state = result.get_statevector()
plot_bloch_multivector(final_state)

=> State is now on the X axis, means it is in superposition state

### Visualize in Quantum Sphere

In [ ]:
plot_state_qsphere(final_state)

=> Understanding:\
In superposition, state can be in both state |0> or |1> at the same time. When we measure, state collaps and provide us any one outcome. In Superposition, probablities of outcome for 0 and 1 is the same.

## Question - 02
You have to create a circuit with the gates CCX, H, SX (√X gate) in such a way that\
you always have roughly  ¼ probability to get state |1> from the target qubit of CCX.\
Create a new Unitary gate with the above mentioned gates according to your implementation and apply to the circuit.\
Now, alter the circuit with Y basis measurement and explain the differences in outcome briefly.

### Constructing circuit

In [ ]:
circuit = QuantumCircuit(3, 1)

circuit.h(0)
circuit.sx(1)
circuit.ccx(0, 1, 2)

circuit.measure(2, 0)

circuit.draw('mpl')

### Testing on Simulator, outcome State 1 from the target qubit will be near 25%

In [ ]:
job = AerSimulator().run(circuit, shots=1000)
result = job.result()
counts = result.get_counts()
plot_histogram(counts)

=> Outcome from the target qubit is nearly 25%, so the circuit construction is complete

### Extracting the gate sequence in as a custom Unitary Gate

In [ ]:
circuit = QuantumCircuit(3)

circuit.h(0)
circuit.sx(1)
circuit.ccx(0, 1, 2)

custom_gate = circuit.to_gate(label="CustomUnitary")

### Creating another circuit with the Custom Gate for testing

In [ ]:
circuit = QuantumCircuit(3, 1)
circuit.append(custom_gate, [0, 1, 2])

circuit.measure(2, 0)

circuit.draw('mpl')

In [ ]:
simulator = AerSimulator()
pass_manager = generate_preset_pass_manager(optimization_level=3, backend=simulator)
isa_circuit = pass_manager.run(circuit)
job = simulator.run(isa_circuit, shots=1000)
result = job.result()
counts = result.get_counts()
plot_histogram(counts)

=> Outcome from the target qubit is nearly 25%, so circuit is working according to the instruction.

### Y - basis Measurement

In [ ]:
circuit = QuantumCircuit(3, 1)
circuit.append(custom_gate, [0, 1, 2])

# preaparing for y basis measurement
circuit.sdg(2)
circuit.h(2)
circuit.measure(2, 0)

circuit.draw('mpl')

In [ ]:
simulator = AerSimulator()
pass_manager = generate_preset_pass_manager(optimization_level=3, backend=simulator)
isa_circuit = pass_manager.run(circuit)
job = simulator.run(isa_circuit, shots=1000)
result = job.result()
counts = result.get_counts()
plot_histogram(counts)

=> We initialize the circuit on z basis. But here we are measuring the circuit on y basis. According to the Heisenberg Uncertainty Principle, if we initialize the circuit in a basis and measure on the other basis, the outcome will be uncertain.\
Here, we initialize the circuit as the outcome for 1 should be nearly 25% but we get around 50%, because of this uncertainty principle.

## Question - 03
Calculate the unitary for the circuit (both Z and Y basis measurement) created in problem 2.\
Explain the differences in outcome and your understanding briefly.

### Unitary Matrix for Z Basis

In [ ]:
circuit = QuantumCircuit(3)

circuit.h(0)
circuit.sx(1)
circuit.ccx(0, 1, 2)

unitary = Operator(circuit).data
print("Unitary matrix for the circuit:")
array_to_latex(unitary)

### Unitary Matrix for Y Basis

In [ ]:
circuit = QuantumCircuit(3)

circuit.h(0)
circuit.sx(1)
circuit.ccx(0, 1, 2)

circuit.sdg([0, 1, 2])
circuit.h([0, 1, 2])

unitary = Operator(circuit).data
print("Unitary matrix for the circuit:")
array_to_latex(unitary)

## Question - 04
Create an entanglement circuit with H, CX gate and apply measurement.\
Run on both a simulator and real quantum device.\
Use proper error mitigation techniques for the real backend (level 1 or level 2).\
Compare the results you obtained by plotting histograms.\
Explain your understanding in detail.

### Entanglement Circuit

In [ ]:
circuit = QuantumCircuit(2)

circuit.h(0)
circuit.cx(0, 1)

circuit.measure_all()

circuit.draw("mpl")

### Run on Simulator

In [ ]:
simulator = AerSimulator()
pass_manager = generate_preset_pass_manager(optimization_level=3, backend=simulator)
isa_circuit = pass_manager.run(circuit)
job = simulator.run(isa_circuit, shots=1000)
result = job.result()
counts = result.get_counts()
plot_histogram(counts)

=> As we construct an Entanglement circuit using 2 qubit, the outcome will be same for both qubit, either 00 or 11.

### Run on Real Devices

In [ ]:
service = QiskitRuntimeService()
backend = service.least_busy(simulator=False, operational=True, min_num_qubits=100)
pass_manager = generate_preset_pass_manager(optimization_level=3, backend=backend)
isa_circuit = pass_manager.run(circuit)
isa_circuit.draw("mpl", idle_wires=False)

In [ ]:
sampler = Sampler(backend)
job = sampler.run([isa_circuit], shots=1000)
result = job.result()
result_real = result[0]

In [ ]:
counts_real = result_real.data.meas.get_counts()
plot_histogram(counts_real)

=> as we know real device have some noise and error, thats why we are getting some output like 01 and 10, but the amount is very low. For these error, we need to run our circuit multiple time and consider the output of which sequence is higher than others.

### Applying Error Mitigration Technique

In [ ]:
from calibration import complete_meas_cal, CompleteMeasFitter

In [ ]:
qr = QuantumRegister(2)
meas_calibs, state_labels = complete_meas_cal(qr=qr, circlabel='mcal')

In [ ]:
for circuit in meas_calibs:
  print('Circuit', circuit.name)
  print(circuit)
  print()

In [ ]:
service = QiskitRuntimeService()
backend = service.least_busy(simulator=False, operational=True, min_num_qubits=100)
pass_manager = generate_preset_pass_manager(optimization_level=3, backend=backend)
isa_circuit = pass_manager.run(meas_calibs)

In [ ]:
isa_circuit[0].draw("mpl", idle_wires=False)

In [ ]:
isa_circuit[1].draw("mpl", idle_wires=False)

In [ ]:
isa_circuit[2].draw("mpl", idle_wires=False)

In [ ]:
isa_circuit[3].draw("mpl", idle_wires=False)

In [ ]:
sampler = Sampler(backend)
job = backend.run(isa_circuit, shots=1000)

while job.status() != 'DONE':
    print(f"Job Status: {job.status()}")
    time.sleep(10)  # Check status every 10 seconds

result = job.result()
#cal_result = result[0]

simulator = AerSimulator()
pass_manager = generate_preset_pass_manager(optimization_level=3, backend=simulator)
isa_circuit = pass_manager.run(meas_calibs)
job = simulator.run(isa_circuit, shots=1000, noise_model=noise_model)
cal_result = job.result()

### Retrive Job

In [3]:
import datetime
three_months_ago = datetime.datetime.now() - datetime.timedelta(days=90)

service = QiskitRuntimeService()
jobs_in_last_three_months = service.jobs(created_after=three_months_ago)

jobs_in_last_three_months[:]  # show first three jobs

[<RuntimeJob('cwy43x7tdtng0087m7k0', 'circuit-runner')>,
 <RuntimeJob('cwxtzv7tdtng0087jffg', 'sampler')>,
 <RuntimeJob('cwxtyrkehebg008jn490', 'sampler')>,
 <RuntimeJob('cwxt4h22ac5g008jjv30', 'sampler')>,
 <RuntimeJob('cwxq2kj2ac5g008jj7dg', 'sampler')>,
 <RuntimeJob('cwxpzxq60bqg008pwhw0', 'sampler')>,
 <RuntimeJob('cwxpy5r997wg008y39s0', 'sampler')>,
 <RuntimeJob('cwxpx14tdtng0087hmc0', 'sampler')>,
 <RuntimeJob('cwxpvzq2ac5g008jj630', 'sampler')>,
 <RuntimeJob('cwxpsjetdtng0087hm0g', 'sampler')>]

In [4]:
service = QiskitRuntimeService()

# Get ID of most recent successful job for demonstration.
# This will not work if you've never successfully run a job.
successful_job = next(j for j in service.jobs() if j.status().name == "DONE")
job_id = successful_job.job_id()
print(job_id)

C:\Users\joyde\AppData\Local\Temp\ipykernel_26224\1475236228.py:5: DeprecationWarning: In a future release of qiskit-ibm-runtime no sooner than 3 months after the release date of 0.30.0, RuntimeJob.status() will be returned as a string instead of an instance of `JobStatus`.
  successful_job = next(j for j in service.jobs() if j.status().name == "DONE")


cwxtzv7tdtng0087jffg


In [16]:
# Sampler version calibration
job_id = "cwxtzv7tdtng0087jffg"

In [25]:
#backend version calibration
job_id = "cwy43x7tdtng0087m7k0"
retrieved_job = service.job(job_id)
print(retrieved_job.status().name)

C:\Users\joyde\AppData\Local\Temp\ipykernel_26224\3822463731.py:4: DeprecationWarning: In a future release of qiskit-ibm-runtime no sooner than 3 months after the release date of 0.30.0, RuntimeJob.status() will be returned as a string instead of an instance of `JobStatus`.
  print(retrieved_job.status().name)


QUEUED


In [17]:
retrieved_job = service.job(job_id)

In [18]:
result = retrieved_job.result()
cal_result = result[0]

In [24]:
cal_result.data

DataBin(c1=BitArray(<shape=(), num_shots=1000, num_bits=2>), meas=BitArray(<shape=(), num_shots=1000, num_bits=2>))

In [ ]:
noisy_counts = cal_result.data.meas.get_counts()
plot_histogram(noisy_counts)

In [ ]:
cal_result = retrieved_job.result()

In [ ]:
from qiskit.result import BaseReadoutMitigator

### Trial

In [ ]:
from qiskit import *
from qiskit import QuantumCircuit
from qiskit.visualization import plot_state_qsphere, plot_histogram, plot_bloch_multivector, array_to_latex
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit.quantum_info import Operator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from math import sqrt
import random
from collections import Counter
import time

QiskitRuntimeService.save_account(channel="ibm_quantum",
                                  token="11318f27c420947d9fcc114bdf0dc0103873ca807db5e77f68463decf6fef341ea99779afdbf5c13cc9d87c68cffb3c8011cb4a1a1165f10d62f87eaee3cee4f",
                                  overwrite=True,
                                  set_as_default=True)

from calibration import complete_meas_cal, CompleteMeasFitter


qr = QuantumRegister(2)
meas_calibs, state_labels = complete_meas_cal(qr=qr, circlabel='mcal')


service = QiskitRuntimeService()

job_id = "cwxtzv7tdtng0087jffg"
retrieved_job = service.job(job_id)

cal_result = retrieved_job.result()

meas_fitter = CompleteMeasFitter(cal_result, state_labels, circlabel='mcal')
array_to_latex(meas_fitter.cal_matrix)

### Resume Error Mitigation

In [ ]:
cal_result = result
meas_fitter = CompleteMeasFitter(cal_result, state_labels, circlabel='mcal')
array_to_latex(meas_fitter.cal_matrix)

In [ ]:
circuit = QuantumCircuit(2)

circuit.h(0)
circuit.cx(0, 1)

circuit.measure_all()

circuit.draw("mpl")

In [ ]:
service = QiskitRuntimeService()
backend = service.least_busy(simulator=False, operational=True, min_num_qubits=100)
pass_manager = generate_preset_pass_manager(optimization_level=3, backend=backend)
isa_circuit = pass_manager.run(circuit)
isa_circuit.draw("mpl", idle_wires=False)

In [ ]:
sampler = Sampler(backend)
job = sampler.run([isa_circuit], shots=1000)
result = job.result()
noisy_results = result[0]

In [ ]:
noisy_counts = noisy_results.data.meas.get_counts()
plot_histogram(noisy_counts)

general_circuit = QuantumCircuit(2, 2)
general_circuit.h(0)
general_circuit.cx(0, 1)
general_circuit.measure([0, 1], [0, 1])

simulator = AerSimulator()
pass_manager = generate_preset_pass_manager(optimization_level=3, backend=simulator)
isa_circuit = pass_manager.run(general_circuit)
job = simulator.run(isa_circuit, shots=1000, noise_model=noise_model)
noisy_results = job.result()
noisy_counts = noisy_results.get_counts()
plot_histogram(noisy_counts)

In [ ]:
# get the fitter object
meas_fitter = meas_fitter.filter

# results with mitigation
mitigated_results = meas_fitter.apply(noisy_results)
mitigated_counts = mitigated_results.get_counts()

In [ ]:
plot_histogram([noisy_counts, mitigated_counts], legend=['noisy', 'mitigated'])

=> this is the combined graph without error mitigation and with error mitigation

## Question - 05
Show Heisenberg uncertainty principle in action for X, Y and Z basis measurement.\
Explain your understanding in detail with proper visualization.\
Initialize a state in Y basis and measure in X,Y and Z basis.\
Show how Error Correction works with any of your preferred circuit.

### Heisenberg Uncertainty Principle

In [ ]:
circuit = QuantumCircuit(1, 1)

# Intializing circuit with |1> state in Z basis
circuit.initialize([0, 1], 0)

# measuring the state on z basis
circuit.measure(0, 0)

circuit.draw('mpl')

In [ ]:
simulator = AerSimulator()
pass_manager = generate_preset_pass_manager(optimization_level=3, backend=simulator)
isa_circuit = pass_manager.run(circuit)
job = simulator.run(isa_circuit, shots=1000)
result = job.result()
counts = result.get_counts()
plot_histogram(counts)

=> as the circuit is initialized on z basis so measing on z basisi will be certain as we initialized.

In [ ]:
circuit = QuantumCircuit(1, 1)

# Intializing circuit with |1> state in Z basis
circuit.initialize([0, 1], 0)

#measuring the state on x basis
circuit.h(0)
circuit.measure(0, 0)

circuit.draw('mpl')

In [ ]:
simulator = AerSimulator()
pass_manager = generate_preset_pass_manager(optimization_level=3, backend=simulator)
isa_circuit = pass_manager.run(circuit)
job = simulator.run(isa_circuit, shots=1000)
result = job.result()
counts = result.get_counts()
plot_histogram(counts)

=> according to Heisenberg Uncertainty Principle, if we initialize in a basis and measure on another basis, outcome will be uncertain.

In [ ]:
circuit = QuantumCircuit(1, 1)

# Intializing circuit with |1> state in Z basis
circuit.initialize([0, 1], 0)

#measuring the state on y basis
circuit.sdg(0)
circuit.h(0)
circuit.measure(0, 0)

circuit.draw('mpl')

In [ ]:
simulator = AerSimulator()
pass_manager = generate_preset_pass_manager(optimization_level=3, backend=simulator)
isa_circuit = pass_manager.run(circuit)
job = simulator.run(isa_circuit, shots=1000)
result = job.result()
counts = result.get_counts()
plot_histogram(counts)

=> according to Heisenberg Uncertainty Principle, if we initialize in a basis and measure on another basis, outcome will be uncertain.

### Initializing state in y basis

In [ ]:
circuit = QuantumCircuit(1, 1)

# initializing circuit in y basis with |+i> state
circuit.initialize([1/sqrt(2), 1j/sqrt(2)], 0)

#measuring the state on y basis
circuit.sdg(0)
circuit.h(0)
circuit.measure(0, 0)

circuit.draw('mpl')

In [ ]:
simulator = AerSimulator()
pass_manager = generate_preset_pass_manager(optimization_level=3, backend=simulator)
isa_circuit = pass_manager.run(circuit)
job = simulator.run(isa_circuit, shots=1000)
result = job.result()
counts = result.get_counts()
plot_histogram(counts)

=> Result is certain, as we measure on the same basis

In [ ]:
circuit = QuantumCircuit(1, 1)

# initializing circuit in y basis with |+i> state
circuit.initialize([1/sqrt(2), 1j/sqrt(2)], 0)

#measuring the state on x basis
circuit.h(0)
circuit.measure(0, 0)

circuit.draw('mpl')

In [ ]:
simulator = AerSimulator()
pass_manager = generate_preset_pass_manager(optimization_level=3, backend=simulator)
isa_circuit = pass_manager.run(circuit)
job = simulator.run(isa_circuit, shots=1000)
result = job.result()
counts = result.get_counts()
plot_histogram(counts)

=> according to Heisenberg Uncertainty Principle, if we initialize in a basis and measure on another basis, outcome will be uncertain.

In [ ]:
circuit = QuantumCircuit(1, 1)

# initializing circuit in y basis with |+i> state
circuit.initialize([1/sqrt(2), 1j/sqrt(2)], 0)

#measuring the state on z basis
circuit.measure(0, 0)

circuit.draw('mpl')

In [ ]:
simulator = AerSimulator()
pass_manager = generate_preset_pass_manager(optimization_level=3, backend=simulator)
isa_circuit = pass_manager.run(circuit)
job = simulator.run(isa_circuit, shots=1000)
result = job.result()
counts = result.get_counts()
plot_histogram(counts)

=> according to Heisenberg Uncertainty Principle, if we initialize in a basis and measure on another basis, outcome will be uncertain.

### Error Correction using a simple circuit

#### Run on Simulator without any error

In [ ]:
circuit = QuantumCircuit(1)

# initializing circuit in |1> state
circuit.x(0)

circuit.measure_all()

circuit.draw('mpl')

In [ ]:
simulator = AerSimulator()
pass_manager = generate_preset_pass_manager(optimization_level=3, backend=simulator)
isa_circuit = pass_manager.run(circuit)
job = simulator.run(isa_circuit, shots=1000)
result = job.result()
counts = result.get_counts()
plot_histogram(counts)

=> this should be the output for this circuit

#### Run on simulator with error

In [ ]:
def constructCircuit():
    circuit = QuantumCircuit(1)

    # initializing circuit in |1> state
    circuit.x(0)

    
    # Apply random bit flip with 10% probability
    if random.random() < 0.1:
        circuit.x(0)
    
    circuit.measure_all()
    
    return circuit

In [ ]:
simulator = AerSimulator()
pass_manager = generate_preset_pass_manager(optimization_level=3, backend=simulator)

all_counts_error = Counter()

for i in range(1000):
    circuit = constructCircuit()
    isa_circuit = pass_manager.run(circuit)
    job = simulator.run(isa_circuit, shots=1)
    result = job.result()
    counts = result.get_counts()
    all_counts_error.update(counts)

plot_histogram(all_counts_error)

=> we initialize the circuit with 1 state but we got some 0 state because of random error 10%

#### Run on simulator with error correction using repeatation code

In [ ]:
def constructCircuit():
    circuit = QuantumCircuit(5, 1)

    # initializing circuit in |1> state
    circuit.x(0)

    # Encoding
    circuit.cx(0, 1)
    circuit.cx(0, 2)

    # Apply random bit flip with 10% probability
    if random.random() < 0.1:
        circuit.x(random.randint(0, 2))

    # syndrome generate
    circuit.cx(0, 3)
    circuit.cx(1, 3)
    circuit.cx(1, 4)
    circuit.cx(2, 4)

    # Recovery Protocol
    circuit.cx(3, 0)
    circuit.cx(4, 2)
    circuit.ccx(4, 3, 0)
    circuit.ccx(4, 3, 2)
    circuit.ccx(4, 3, 1)

    circuit.measure(0, 0)
    
    return circuit

In [ ]:
simulator = AerSimulator()
pass_manager = generate_preset_pass_manager(optimization_level=3, backend=simulator)

all_counts_error_corrected = Counter()

for i in range(1000):
    circuit = constructCircuit()
    isa_circuit = pass_manager.run(circuit)
    job = simulator.run(isa_circuit, shots=1)
    result = job.result()
    counts = result.get_counts()
    all_counts_error_corrected.update(counts)

plot_histogram(all_counts_error_corrected)

=> Here you can see all the error are corrected using repeatation code

In [ ]:
plot_histogram([all_counts_error, all_counts_error_corrected], legend=['with Error', 'Error Corrected'])

=> compare of both result without error correction and with error correction